In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import re

In [5]:
app_train = pd.read_csv("data/train.csv")
app_test = pd.read_csv("data/test.csv")
sample = pd.read_csv("data/simple_submission.csv")

In [6]:
print('Training data shape: ', app_train.shape)
app_train.head()

Training data shape:  (53030, 195)


,id,label,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,...,partner5_H,partner5_K,partner5_L,brief,num_of_phone,Field_78,Field_79,Field_80,Field_81,Field_82
0,0,1,2019-07-24T03:39:02.854Z,2019-07-31T20:10:02Z,1.0,GH,2018-12-27,2018-12-27,2019-01-01,2019-07-31,...,0.0,0.0,0.0,cb1,1.0,NaN,NaN,NaN,NaN,1
1,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,4,1.0,NaN,NaN,NaN,NaN,1
2,2,0,2019-01-17T07:15:26.367Z,2019-01-17T07:17:45Z,2.0,T1,2019-01-17,2019-01-17,2019-01-01,2019-12-31,...,0.0,0.0,0.0,1,1.0,33.0,10.769445,6.466667,0.0,2
3,3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,1.0,33.0,10.769445,6.466667,0.0,3
4,4,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1,1.0,33.0,10.769445,6.466667,0.0,3


In [7]:
print('Training data shape: ', app_test.shape)
app_test.head()

Training data shape:  (20381, 194)


,id,Field_1,Field_2,Field_3,Field_4,Field_5,Field_6,Field_7,Field_8,Field_9,...,partner5_H,partner5_K,partner5_L,brief,num_of_phone,Field_78,Field_79,Field_80,Field_81,Field_82
0,53030,2019-10-14T01:37:41.959Z,2019-10-14T01:37:41.959Z,2.0,T1,8/27/2019,8/27/2019,9/24/2019,9/23/2020,8/27/2019,...,0.0,0.0,0.0,NaN,2.0,NaN,NaN,NaN,NaN,1
1,53031,2019-09-10T09:20:40.066Z,2019-09-11T07:53:44Z,2.0,T1,9/10/2019,9/10/2019,9/14/2019,9/13/2020,9/10/2019,...,0.0,0.0,0.0,notfound,1.0,NaN,NaN,NaN,NaN,1
2,53032,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0,1.0,NaN,NaN,NaN,NaN,1
3,53033,2019-10-21T03:29:30.074Z,2019-10-21T04:17:26Z,2.0,T1,10/21/2019,10/21/2019,11/15/2019,11/14/2020,10/21/2019,...,0.0,0.0,0.0,NaN,1.0,NaN,NaN,NaN,NaN,1
4,53034,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,3,1.0,27.0,NaN,27.0,27.0,1


#### 1.Examine Missing Values

In [8]:
def missing_values(df):
    mis_val = df.isnull().sum() #tổng các giá trị còn thiếu
    mis_val_per = 100 * df.isnull().sum()/len(df) #Phần trăm các giá trị còn thiếu
    mis_val_table = pd.concat([mis_val, mis_val_per], axis=1) #Tạo thành bảng để thể hiện 2 giá trị trên
    mis_val_table_rename = mis_val_table.rename(
    columns = {0: "Missing", 1 : "Percent %"})
    #Sắp xếp bảng theo tỉ lệ phần trăm giảm dần
    mis_val_table_rename = mis_val_table_rename[mis_val_table_rename.iloc[:, 1] != 0].sort_values("Percent %", ascending=False).round(1)
    print("Có tất cả " + str(df.shape[1]) + " cột.\n"
          "Trog đó có " + str(mis_val_table_rename.shape[0]) + " cột bị thiếu data")
    return mis_val_table_rename

In [9]:
missing_values = missing_values(app_train)
missing_values

Có tất cả 195 cột.
Trog đó có 192 cột bị thiếu data


,Missing,Percent %
Field_35,46198,87.1
Field_12,45650,86.1
Field_11,45616,86.0
Field_18,44643,84.2
maCv,43882,82.7
...,...,...
partner1_K,4747,9.0
partner1_H,4747,9.0
partner0_D,4747,9.0
num_of_phone,2512,4.7


Khi đến lúc xây dựng các mô hình học máy của mình, chúng ta sẽ phải điền vào các giá trị còn thiếu này (được gọi là áp đặt). Trong công việc sau này, chúng tôi sẽ sử dụng các mô hình như XGBoost có thể xử lý các giá trị bị thiếu mà không cần áp đặt. Một tùy chọn khác sẽ là loại bỏ các cột có tỷ lệ giá trị bị thiếu cao, mặc dù không thể biết trước liệu các cột này có hữu ích cho mô hình của chúng tôi hay không. Do đó, chúng tôi sẽ giữ tất cả các cột ngay bây giờ.

### 2.Column Types

In [10]:
app_train.dtypes.value_counts()

float64    133
object      59
int64        3
dtype: int64

Số lượng object trong mỗi columns

In [17]:
app = app_train.select_dtypes('object').apply(pd.Series.nunique, axis = 0)
app

Field_1                   21506
Field_2                   18667
Field_4                       2
Field_5                    1901
Field_6                    1870
Field_7                    1199
Field_8                    1392
Field_9                    1929
Field_11                    836
Field_12                      7
Field_15                   1539
Field_18                   6587
Field_25                   1613
Field_32                   1527
Field_33                   1753
Field_34                   2635
Field_35                    632
gioiTinh                      2
diaChi                    21057
Field_36                     34
Field_38                      6
Field_40                    541
Field_43                   3605
Field_44                   9901
Field_45                  10957
Field_46                  14311
Field_47                      5
Field_48                  12650
Field_49                  12818
Field_54                     44
Field_55                    220
Field_56

In [16]:
from sklearn.preprocessing import LabelEncoder
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtypes == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 10:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

TypeError: Encoders require their input to be uniformly strings or numbers. Got ['float', 'str']